<div align="center"; span style="color:#336699"><b><h2> Algorithm Workflow </h2></b></div>
<hr style="border:2px solid #0077b9;">
<br/>
<div style="text-align: center;font-size: 90%;">
    Helvécio B. Leal Neto, <sup><a href="https://orcid.org/0000-0002-7526-2094"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    Alan J. P. Calheiros<sup><a href="https://orcid.org/0000-0002-7526-2094"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
   :<br/><br/>
 :  National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:helvecio.neto@inpe.br">helvecio.neto@inpe.br</a>, <a href="mailto:alan.calheiros@inpe.br">alan.calheiros@inpe.br</a>
    <br/><br/>
    Last Update: Nov 1, 2024
</div>
<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> This Jupyter notebook demonstrates the complete processing flow of the tracking process of pyForTraCC algorithm. There you can check the example of tracking using radar data.
</div>    
<br/>
<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook was based on the <a href="https://github.com/fortracc-project/pyfortracc">"pyfortracc"</a> example from the pyFortracc:</b>
    <div style="margin-left: 10px; margin-right: 10px; margin-top:10px">
      <p> Leal Neto, H.B.; Calheiros, A.J.P.;  pyForTraCC Algorithm. São José dos Campos, INPE, 2024. <a href="https://github.com/fortracc-project/pyfortracc" target="_blank"> Online </a>. </p>
    </div>
</div>

### Schedule

 [1. Installation](#install)<br>
 [2. Radar Input Data](#data)<br>
 [3. Read Function](#read)<br>
 [4. Tracking Parameters](#parameters)<br>
 [5. Algorithm Workflow](#parameters)<br>
 [- Features Extraction](#features)<br>
 [- Spatial Operations](#spatial)<br>
 [- Cluster Link](#link)<br>
 [6. Explore Output](#output)

<a id='install'></a>
#### 1. Installation

Installing the pyFortraCC package can be done using the pip install command. 

All dependencies will be installed in the current Python environment and the code will be ready to use.

In [ ]:
# Install directly from github
!pip3 install --upgrade git+https://github.com/fortracc/pyfortracc.git@main#egg=pyfortracc &> /dev/null

<a id='data'></a>
#### 1. Radar Input Data

The data in this example corresponds to a small sample of scans from the S-Band Radar located in the city of Manaus-AM Brazil.<br>
 Data processed and published by Schumacher, Courtney and Funk, Aaron (2018) were separated, <br>
 and are available in full on the ARM platform https://www.arm.gov/research/campaigns/amf2014goamazon.<br>
 This data is part of the GoAmazon2014/5 project and is named "Three-dimensional Gridded S-band Reflectivity and Radial Velocity<br>
 from the SIPAM Manaus S-band Radar dataset".<br>
https://doi.org/10.5439/1459573


In [ ]:
# Download the exemple input files
!pip3 install --upgrade --no-cache-dir gdown &> /dev/null
!gdown 'https://drive.google.com/uc?id=1UVVsLCNnsmk7_wOzVrv4H7WHW0sz8spg'
!unzip -qq -o input.zip
!rm -rf input.zip

<a id='read'></a>
#### 3. Read Function

The downloaded data is compressed with the .gz extension, however, it is of the netCDF4 type. The variable that represents reflectivity is DBZc. This data also has multiple elevations, and we arbitrarily chose elevation 5, which corresponds to the volumetric scan level at 2.5 km height. We extract the data and apply a NaN value to the data mask -9999.

In [ ]:
# Define the Read function
import gzip
import netCDF4
import numpy as np

def read_function(path):
    variable = "DBZc"
    z_level = 5 # Elevation level 2.5 km
    with gzip.open(path) as gz:
        with netCDF4.Dataset("dummy", mode="r", memory=gz.read()) as nc:
            data = nc.variables[variable][:].data[0,z_level, :, :][::-1, :]
            data[data == -9999] = np.nan
    gz.close()
    return data

To visualize the data, we use an function from pyFortraCC that reads the data and create an animation of the radar scan.

In [ ]:
from pyfortracc import plot_animation

In [ ]:
# Visualize the data using the plot_animation function
plot_animation(path_files='input/*.gz', # Path to the files
                          read_function=read_function, # Read function
                          num_frames=10, min_val=0, max_val=60, # Number of frames and maximum value
                          cbar_title='dBZ') # Colorbar title

<a id='parameters'></a>
#### 4. Tracking Parameters

For this example, we will track reflectivity clusters at multiple thresholds and sizes <br>Arbitrarily selecting thresholds of 20, 30 and 35 dBZ with clusters of 5,4 and 3 pixels <br>of minimum size. The segmentation operator will be >=, that is, the clusters will be <br>segmented in order of greatest equal for each threshold and the delta of the observations <br>is 12 minutes.<br>
To improve the precision of the tracking, we will use the DBSCAN algorithm with a elpsilon with 3 pixels,<br>
this method considers the spatial distance pixels around the cluster and includes them in the same cluster.


In [ ]:
# Example Name list dictionary of mandatory parameters
name_list = {}
name_list['input_path'] = 'input/' # path to the input data
name_list['output_path'] = 'output/' # path to the output data
name_list['timestamp_pattern'] = 'sbmn_cappi_%Y%m%d_%H%M.nc.gz' # timestamp file pattern
name_list['thresholds'] = [20,30,35] # in dbz
name_list['min_cluster_size'] = [3,3,3] # in number of points per cluster
name_list['operator'] = '>=' # '>= - <=' or '=='
name_list['delta_time'] = 12 # in minutes
name_list['min_overlap'] = 20 # Minimum overlap between clusters in percentage

# Optional parameters, if not set, the algorithm will not use geospatial information
name_list['lon_min'] = -62.1475 # Min longitude of data in degrees
name_list['lon_max'] = -57.8461 # Max longitude of data in degrees
name_list['lat_min'] = -5.3048 # Min latitude of data in degrees
name_list['lat_max'] = -0.9912 # Max latitude of data in degrees

name_list['cluster_method'] = 'dbscan' # DBSCAN Clustering method
name_list['eps'] = 3 # in pixels

<a id='workflow'></a>
#### 5. Algorithm Workflow

In this example we will use the modules separately, that is, each internal module of the pyForTraCC algorithm will be called individually.<br>
The track workflow is divided into four modules:
<ol>
  <li><b>Feature detection</b>: Focuses on identifying distinct characteristics for precise tracking.
  </li>
  <li><b>Spatial Operations</b>: Involves spatial operations (intersection, union, difference, etc) between the features of consecutive time steps (t and t-1).
  <li><b>Trajectory Linking</b>: Passing one by one the time steps, the algorithm link the features of consecutive time steps based on the association created in the previous step. The algorithm create a trajectory for each feature.
  <li><b>Concatenate</b>:
  </li>
</ol>

##### - Features Extraction
Image processing strategies were combined with clustering and rasterization algorithms to achieve the results obtained by the extract_features function. The following sequence is present in the implementation of the algorithm:

1. Read the file using the read_funcion.
2. Image segmentation according to each threshold.
3. Labeling of clusters. Two clustering options are implemented (DBSCAN and ndimage). 
5. Vectorization of the clusters using rasterio.features.shapes to acquire the boundary polygon and the centroid of the clusters.

<span> <img src="./img/features.png" style="height:320px;" align="left"></span>

In [ ]:
from pyfortracc import features_extraction

# Note: If you are running this notebook using MacOS, you may need to set parallel=False.
features_extraction(name_list, read_function, parallel=True)

For each of submodules of pyForTraCC, the algorithm returns a dataframe into a parquet file. 
The dataframe for features extraction contains the following columns:

- timestamp: The timestamp of the radar scan, based on name_list['timestamp_pattern'].
- cluster_id: The cluster identification by clusterization algorithm.
- threshold_level: The threshold level of the cluster.
- threshold: The threshold value of the cluster.
- size: The size of the cluster (in pixels).
- min max mean std: The minimum, maximum, mean and standard deviation of values of the cluster.
- array_values: The values of the cluster.
- array_x array_y: The x and y coordinates of the cluster.
- geometry: The boundary polygon of the cluster.
- file: The file name of the radar scan.

In [ ]:
import glob
import pandas as pd

features = sorted(glob.glob(name_list['output_path'] + '/track/processing/features/*.parquet'))
features = pd.concat(pd.read_parquet(f) for f in features)
display(features.head(5))

##### - Spatial operations 

Spatial operations are fundamental in Geographic Information Systems (GIS) and spatial databases. And the Geopandas implementation simplifies the process by combining GeoDataframes stored in .parquet files by features extraction module. Based on their spatial relationships between clusters of current and previous times, we use the Spatial Join to create associations between boundary clusters of consecutive times. The algoritmo use this method performs to various types of spatial joins such as overlays, within, contains.

To demonstrate the basic functioning of the spatial operations mode, we will use as a base two consecutive times listed here as the variables cur_frame and prev_frame, which store information about the characteristics of the systems for each time.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.wkt import loads

In [ ]:
# Read the features parquet files
cur_frame = pd.read_parquet('./output/track/processing/features/20140816_1012.parquet')
prev_frame = pd.read_parquet('./output/track/processing/features/20140816_1000.parquet')
cur_frame['geometry'] = cur_frame['geometry'].apply(loads)
prev_frame['geometry'] = prev_frame['geometry'].apply(loads)
# Convert to geo dataframes where column is geometry
cur_frame = gpd.GeoDataFrame(cur_frame)
prev_frame = gpd.GeoDataFrame(prev_frame)

In figure below, we can see the spatial operations between the clusters of two consecutive times. The Red polygons represent the clusters of the current time and the blue polygons represent the clusters of the previous time. And its possible to see these clusters have a spatial relationship between them.

In [ ]:
# Visualize the features and see have a visual overlap between the geometries
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
cur_frame.boundary.plot(ax=ax, color='red', alpha=0.5, label='Current Frame')
prev_frame.boundary.plot(ax=ax, color='blue', alpha=0.5, label='Previous Frame')
plt.legend( loc='upper left')

To demonstrate one of the operations performed in the algorithm, below is the GeoPandas sjoin function that checks the overlaps between the two GeoDataframes. 
Note that the return of the function will be another GeoDataframe, however only the index and index_right columns are of interest to us, 
as in these columns we have the information we need to make the associations between the geometries of the consecutive time clusters.
In addition to the overlap operation, there are several others that can be seen at:
https://shapely.readthedocs.io/en/latest/manual.html#binary-predicates

The result of overlaps is stored into a dataframe containing the indexes of the clusters of the current and previous times that have a spatial relationship between them.

In [ ]:
# Perform the spatial join between the previous and current frame
overlaps = gpd.sjoin(cur_frame, prev_frame, how='inner',predicate='overlaps')[['index_right']].reset_index()
# Ex: index column represents the index of the current frame and index_right represents the index of the previous frame
overlaps.head()

In addition, the spatial operations module also has additional vector extraction methods. These methods are covered in the work https://doi.org/10.3390/rs14215408

To activate the methods just add the flags to the name_list.

In [ ]:
# Add correction methods
name_list['spl_correction'] = True # Perform the correction at Splitting events
name_list['mrg_correction'] = True # Perform the correction at Merging events
name_list['inc_correction'] = True # Perform the correction using Inner Core vectors
name_list['opt_correction'] = True # Perform the correction using the Optical Flow method (New Method)
name_list['elp_correction'] = True # Perform the correction using the Ellipse method (New Method)
name_list['validation'] = True # It is used to perform the validation of the correction methods

To run the spatial operations module, we need to call the function spatial_operations and pass the name_list as a parameter.

In [ ]:
from pyfortracc import spatial_operations

# (Note: If you are running this notebook using MacOS, you may need to set parallel=False)
spatial_operations(name_list, read_function, parallel=True)

The dataframe for spatial operations contains the following columns:

- status: The status of the spatial operation, possible values are: NEW, CON, MRG, SPL, SPL/MRG
- threshold_level: The threshold level of the cluster.
- inside_clusters: Number of clusters inside the current threshold level cluster.
- past_idx: The index of the cluster in the previous time.
- inside_idx: The index of the cluster inside the current cluster.
- merge_idx: The index of the cluster merged at previous time by the current cluster.
- split_pr_idx: The index of the cluster split at previous time by the current cluster.
- split_cr_idx: The index of the cluster split at current time by the previous cluster.
- overlap: The overlap area between the clusters.
- within: If is a within cluster. (True or False)
- contains: If is a contains cluster. (True or False)
- board: If is a cluster at the board. (True or False)
- board_idx: The index of the cluster at the other side of the board.
- u_: The u component of the cluster (in pixels or degrees, depending if run using geographic coordinates).
- v_: The v component of the cluster (in pixels or degrees, depending if run using geographic coordinates).
- trajectory: The trajectory of the cluster (LineString).
- vector_field: The vector field of the cluster (MultiLineString).
- expansion: The expansion rate between the clusters of consecutive times.
- u_spl: The u component of the cluster by Split Correction.
- v_spl: The v component of the cluster by Split Correction.
- u_mrg: The u component of the cluster by Merge Correction.
- v_mrg The v component of the cluster by Merge Correction.
- u_inc: The u component of the cluster by Inner Cores Correction.
- v_inc: The v component of the cluster by Inner Cores Correction.
- u_opt: The u component of the cluster by Optical Flow Correction.
- v_opt: The v component of the cluster by Optical Flow Correction.
- u_elp: The u component of the cluster by Elliptical Correction.
- v_elp: The v component of the cluster by Elliptical Correction.
- u_noc: The u component of the cluster by No Correction.
- v_noc: The v component of the cluster by No Correction.
- far: The False Alarm Rate of method, if the validation is True into a name_list.
- method: The best method of correction, if the validation is True into a name_list.

In [ ]:
# Read the spatial parquet files
spatial = sorted(glob.glob(name_list['output_path'] + '/track/processing/spatial/*.parquet'))
spatial_df = pd.concat(pd.read_parquet(f) for f in spatial)
display(spatial_df.tail())

##### Cluster Link

The cluster connection module makes the association between consecutive time tables by associating the cluster indices that were identified by the spatial operations module.

In [ ]:
from pyfortracc import cluster_linking

# This module only works in serial mode
cluster_linking(name_list)

Bellow we show how cluster link works. Set two spatial parquets from consecutive timestamps. And show the association between them based on indexes.

In [ ]:
# Read current and previous frames parquet files
cur_frame = pd.read_parquet('output/track/processing/spatial/20140816_1212.parquet')
prv_frame = pd.read_parquet('output/track/processing/spatial/20140816_1200.parquet')

The association is made by the indexes of the clusters of the current and previous times that have a spatial relationship between them. The algorithm uses the indexes of the clusters that have a spatial relationship between them to create a trajectory for each cluster.


In [ ]:
# The association uses the column 'past_idx' to link the previous frame with the current frame
cur_frame[['past_idx']].head()

In [ ]:
# The association is performed by linking the 'past_idx' column with the index of the previous frame
prv_frame.loc[cur_frame['past_idx'].dropna().astype(int).values].head()

When the link is made, the algorithm creates Unique IDs (uid) for the same cluster in different times. The uid is a unique identifier for each cluster that is maintained throughout the tracking process. For clusters of threshold level 0, the uid is a integer number, for inner clusters, the uid is called iuid, and have a integer part of the uid and a decimal part that represents the inner cluster. The uid and iuid is a unique identifier for each cluster that is maintained throughout the tracking process. Additionally, the algorithm creates a trajectory for each cluster and a lifetime of the cluster.

In [ ]:
# Show the linking results
linked = sorted(glob.glob(name_list['output_path'] + '/track/processing/linked/*.parquet'))
linking_df = pd.concat(pd.read_parquet(f) for f in linked)
linking_df

#### Concatenate 

To simplify the process of running the algorithm, we have created a function that concatenates the results of the previous modules. The function receives the name_list as a parameter and creates a single parquet file for each timestamp of track process.

In [ ]:
from pyfortracc import concat

# Concatenate the features, spatial and linking dataframes into a single dataframe
concat(name_list)

The output of concatenate is a entity called tracking table. The tracking table is the generalized output  located in the output directory of name_list['output'] ('output_path/trackingtable'). The information obtained in the tracking process is stored in a tabular format, and is organized according to the tracking time. Listed below are the names of the columns (output variables) and what they represent.

- Each row of tracking table is related to a cluster at its corresponding threshold level. 
- The information spans from unique identifiers and descriptive statistics to geometric properties and temporal features. 
- The Tracking table structure provides a comprehensive view of grouped entities, facilitating analysis and understanding of patterns across different threshold levels.

Tracking table columns:

*   **timestamp** (datetime64[us]): Temporal information of cluster.
*   **uid** (float64): Unique idetifier of cluster.
*   **iuid** (float64): Internal Unique idetifier of cluster.
*   **threshold_level** (int64): Level of threshold.
*   **threshold** (float64): Specific threshold.
*   **status** (object): Entity status (NEW, CONTINUOUS, SPLIT, MERGE, SPLIT/MERGE)
*   **u_, v_** (float64): Vector components.
*   **inside_clusters** (object): Number of inside clusters.
*   **size** (int64): Cluster size in pixels.
*   **min, mean, max, std** (float64): Descriptive statistics.
*   **delta_time** (timedelta64[us]): Temporal variation.
*   **file** (object): Associated file name.
*   **array_y, array_x** (object): Cluster array coordinates.
*   **vector_field** (object): Associated vector field.
*   **trajectory** (object): Cluster's trajectory.
*   **geometry** (object):  Boundary geometric representation of the cluster.
*   **lifetime** (int64): Cluster lifespan in minutes.
*   **duration** (int64): Cluster duration in minutes.
*   **genesis** (int64): Cluster genesis, with genesis: 1, active: 0, and death: -1.

In [ ]:
# The tracking table is saved in the output path
tracking_files = sorted(glob.glob(name_list['output_path'] + '/track/trackingtable/*.parquet'))
tracking_table = pd.concat(pd.read_parquet(f) for f in tracking_files)
display(tracking_table.head())

#### Resumed track Function

All the process showed above is resumed in the track function. The track function is the main function of the pyForTraCC algorithm. It receives the name_list as a parameter and runs the entire tracking process. The track function is responsible for calling the internal modules of the algorithm in the correct order, and for creating the tracking table.

In [ ]:
import pyfortracc as pf

# Track function
pf.track(name_list, read_function, parallel=True)

#### Explore the results in tracking table

To explore the results of the tracking process, we can use the tracking table. For this example, we will find the with a max lifetime and explore the results showing the track process using animation.

In [ ]:
# Get two maxlifetime clusters from the track_table
maxlifetime = 2
max_lifetimes = tracking_table.groupby('uid').size().nlargest(maxlifetime).index.values
max_clusters = tracking_table[tracking_table['uid'].isin(max_lifetimes)]
print('The clusters with the highest lifetime are the uids: {}'.format(max_lifetimes))

Visualize the maxlifetime system in the tracking table.

In [ ]:
# Plot the tracking data for periods of time. Note: If the min and max value is a larger time interval, 
# the plot will be slower
plot_animation(read_function=read_function, # Read function
                          name_list=name_list, # Name list dictionary
                          uid_list=max_lifetimes.tolist(), # List of uids
                          start_timestamp = max_clusters['timestamp'].min().strftime('%Y-%m-%d %H:%M:%S'), 
                          end_timestamp= max_clusters['timestamp'].max().strftime('%Y-%m-%d %H:%M:%S'),
                          cbar_title='dBZ', # Colorbar title
                          threshold_list=[20], # Threshold list
                          trajectory=True, # Plot the trajectory
                          max_val=60, # Maximum value for the colorbar
                          min_val=20, # Minimum value for the colorbar
                          info_cols=['uid','method','far'],
                          background='google', # Background type
                          )